In [1]:
!pip install pymupdf

In [3]:
# extract images with pymupdf
import fitz
import os


def extract_images_from_pdf(pdf_path, output_directory):
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page.number+1}_{img_index}.png"
            with open(os.path.join(output_directory, image_filename), "wb") as img_file:
                img_file.write(image_bytes)
            images.append(image_filename)
    return images


output_directory = "../data/processed/images/"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
pdf_path = "../data/raw/Our-Impact-in-2023-pages.pdf"
extracted_images = extract_images_from_pdf(pdf_path, output_directory=output_directory)
print("Extracted Images:", extracted_images)

Extracted Images: []


In [4]:
# extract images with pypdf
from pypdf import PdfReader
import os

output_directory = "../data/processed/images/"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
reader = PdfReader("../data/raw/Our-Impact-in-2023-pages.pdf")
for page in reader.pages:
    for image in page.images:
        with open(os.path.join(output_directory, image.name), "wb") as fp:
            fp.write(image.data)

/Users/lanchu/mambaforge/envs/genAI/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [7]:
!pip install -U langchain-openai
!pip install --upgrade open-interpreter
!pip install pytesseract

In [ ]:
# find relevant images with OCR
import pytesseract
from PIL import Image
import os
from tqdm import tqdm

# Define the global list outside of the function
relevant_images = []


def find_relevant_images_with_ocr(root_dir, keywords):
    global relevant_images  # Use the global keyword to modify the global list
    # Initialize a list to store all the image paths for the progress bar
    image_paths = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".png"):
                image_paths.append(os.path.join(root, file))

    for image_path in tqdm(image_paths, desc="Processing Images"):
        try:
            text = pytesseract.image_to_string(Image.open(image_path))
            if any(keyword in text.lower() for keyword in keywords):
                relevant_images.append(image_path)
        except Exception as e:
            print(f"Error processing file {image_path}: {e}")


# Example usage
root_directory = "../data/validation/adobe_raw/"
keywords = ["job"]
find_relevant_images_with_ocr(root_directory, keywords)
print(relevant_images)


In [10]:
text = pytesseract.image_to_string(
    Image.open("../data/raw/figures/Our-Impact-in-2023-pages.jpg")
)
print(text)

es Pe

CAPs Data Quality Scores (score 1) are used to qualify the level of precision ofthe estimates we disclose pe PCAF asset
lass, Data quality cores are spc to each asst class. Data with PCAF scores 1/2, (ened and unverified respectively)
company evel reported emisions data, represent tha highest data quality and ests inthe most accurate emissions

Freneedensens 0 its I estimates. Tiss tolowedby physical cv based emissions (PCAF sore 3 and PCA scores, given fran economic
se —ce dctviy-basademisionsaproach Westie to nl the bes data ualty score based n data aalabty and fealty,
Deals onthe average data quay soe per porto can be ound in Anne ofthe Amal Report.
a sie

Franced Fane a me iy sini noe

femisions ——emisont “Thanks to increased avalabilty of data and improved methodologies this year we ae able to repr financed emisions fr
po 201 ‘3% oftherelevant (1% of ourtotal assets) on-blance sheet asetsin ou portfolio, covering allofthe most arbor intensive

sector region combinations of 